### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import re
import numpy as np
from math import log
import random
import csv

import nltk 
from nltk.text import Text 
import pymorphy2 as pm2 
pmm = pm2.MorphAnalyzer() 

nltk.download("stopwords") 
from nltk.corpus import stopwords 
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
with open('quora_question_pairs_rus.csv', 'r', encoding = 'utf-8') as f:  
    spamreader = csv.reader(f)
    all_data = [row[1:] for row in spamreader][1:]
    corpus = [row[1] for row in all_data]

In [13]:
k = 2.0
b = 0.75
lengths = [len(doc) for doc in corpus]
avgdl = sum(lengths)/len(lengths)
N = len(corpus)

In [14]:
def q_score(q, N, doc, len_doc):        #подсчёт score отдельного слова
    n = np.count_nonzero(tf_matrix[q])
    tf = len(re.findall(' '+q+' ', doc))
    idf = IDF(N, n)
    score = idf*((tf*(k+1))/(tf+k*(1-b+b*len_doc/avgdl)))
    return score
    
def bm25(query, doc) -> float:   #query подать уже нормализованную))
    doc_contents = doc.split(' ')
    len_doc = len(doc_contents)
    scores = []
    for q in query:
        try:
            scores.append(q_score(q, N, doc, len_doc))
        except KeyError:
            scores.append(0)
    return sum(scores)

def IDF(N, n):   # qi - одно слово из запроса
    idf = log((N-n+0.5)/(n+0.5))
    return idf

In [15]:
def preprocess_query(raw_query):
    normalized = [pmm.normal_forms(x)[0] for x in raw_query.split() if x not in russian_stopwords]
    return normalized

есть файл с заранее преобработанными данными, тут он открывается (на гитхаб выгружен зип архив с этим файлом)

In [16]:
with open('lemmatized.txt', 'r', encoding='utf-8') as f:
    content = f.read()
    preprocessed_docs = content.split('\t')

делаем матрицу, содержащую tf

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(preprocessed_docs[:1000])
tf_matrix = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names())

делаем матрицу, содержащую bm25

In [23]:
bm25_matrix = pd.DataFrame(columns=vectorizer.get_feature_names())
for text in preprocessed_docs[:1000]:
    to_m_scores = []
    for item in vectorizer.get_feature_names():
        if item in text.split():
            len_doc = len(text.split())
            to_m_scores.append(q_score(item, N, text, len_doc))
        else:
            to_m_scores.append(0)
    bm25_matrix = bm25_matrix.append(pd.Series(to_m_scores, index=bm25_matrix.columns ), ignore_index=True)

создаём запрос

In [62]:
def create_query(all_data):
    for_queries = [row[0] for row in all_data[:1000]]
    raw_query = random.choice(for_queries)
    QUERY = preprocess_query(raw_query)
    return QUERY

Способ 1, цикл

In [159]:
def process_one_query(QUERY, bm25_matrix):
    search_results = {}
    
    for index, row in  bm25_matrix.iterrows():
        document_score = []
    
        for word in QUERY:
            if word in vectorizer.get_feature_names():
                document_score.append(row[word])
        
        search_results[index] = sum(document_score)
        
    return search_results

In [87]:
def sort_results(one_query_output):
    results_sorted = sorted(one_query_output, key=one_query_output.get, reverse=True)
    return results_sorted[:5]

In [160]:
def loop_search(bm25_matrix, n_queries):
    p1 = 0
    
    for i in range(n_queries):
        QUERY = create_query(all_data)
        #print(QUERY)
    
        one_query_output = process_one_query(QUERY, bm25_matrix)
        output_sorted = sort_results(one_query_output)
    
        for m in output_sorted:
            #print(all_data[m][1])
            if all_data[m][2] == '1':
                p1 += 1
        
    print('\n\nТочность поиска: ', p1/(n_queries*5))
    

In [162]:
%%time 
loop_search(bm25_matrix, 10)



Точность поиска:  0.34
Wall time: 2min 6s


Способ 2, матрица*вектор

In [94]:
def query_to_vector(QUERY):
    vector = [0] * len(vectorizer.get_feature_names())
    for i, word in enumerate(vectorizer.get_feature_names()):
        if word in QUERY:
            vector[i] = 1
    return vector

In [152]:
def matrix_search(n_queries, bm25_matrix):
    p2 = 0
    
    for i in range(n_queries):
        QUERY = create_query(all_data)
        vector = query_to_vector(QUERY)
        #print('ЗАПРОС       ', QUERY)
    
        result_array = np.dot(bm25_matrix, vector)
        sorted_result_array = sorted([(e,i) for i,e in enumerate(result_array)], reverse = True)
    
        for item in list(sorted_result_array[:5]):
            #print(all_data[item[1]][1])
            if all_data[item[1]][2] == '1':
                p2 += 1
            
    print ('\n\nТочность поиска: ', p2/(n_queries*5))
    

In [153]:
%%time
matrix_search(10, bm25_matrix)



Точность поиска:  0.38
Wall time: 125 ms


### __Задача 2__:    


Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

In [131]:
raw_query_christmas = 'рождественские каникулы'
query_christmas = preprocess_query(raw_query_christmas)

с помощью матрицы*вектор

In [133]:
vector_christmas = query_to_vector(query_christmas)

result_array_christmas  = np.dot(bm25_matrix, vector)
sorted_result_array_christmas  = sorted([(e,i) for i,e in enumerate(result_array_christmas)], reverse = True)
    
for item in list(sorted_result_array_christmas[:10]):
    print(all_data[item[1]][1], item[0])

каков наилучший способ начать робототехнику, которая является лучшей платой развития, на которой я могу начать работать над ней 35.49045999647447
какие из лучших оплачиваемых рабочих мест, которые можно сделать из дома 32.383849208625016
как тринадцать лет, что самое лучшее, что я могу сделать для своего здоровья 31.31617743537609
почему люди, которые сделали магистерские степени в любой области, советуют вам не делать мастер 30.599360849153697
я девушка, и я люблю своего лучшего друга, который тоже девушка, что я должен делать 29.323997633009164
Я девушка в средней школе, мои лучшие друзья - это все девушки, которых я узнал, что я бисексуал, как я могу сказать своему лучшему другу, что девушка без нее волнуется, и это меняет всю нашу дружбу 29.187074397300727
какие хаки в повседневной жизни 23.847245961202173
что я могу сделать, когда мне 18 лет 18.040877830538786
3d rad: как я могу сделать миссию 18.040877830538786
что я могу сделать, чтобы не ревновать кого-то 18.040877830538786


обычный способ

In [136]:
one_query_christmas_output = process_one_query(query_christmas)
#print(one_query_christmas_output)
output_christmas_sorted = sort_results(one_query_christmas_output)


for m in output_christmas_sorted:
    print(all_data[m][1], one_query_christmas_output[m])

что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад 0
как повысить скорость интернета путем взлома через dns 0
найти остаток, когда математика 23 ^ 24 математика разделена на 24 23 0
какая рыба выживет в соленой воде 0
Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне 0


### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1

In [142]:
b15 = 0
b11 = 1

In [144]:
def q_score_b(q, N, doc, len_doc, b):        #подсчёт score отдельного слова
    n = np.count_nonzero(tf_matrix[q])
    tf = len(re.findall(' '+q+' ', doc))
    idf = IDF(N, n)
    score = idf*((tf*(k+1))/(tf+k*(1-b+b*len_doc/avgdl)))
    return score

def bm25_b(query, doc, b) -> float:   #query подать уже нормализованную))
    doc_contents = doc.split(' ')
    len_doc = len(doc_contents)
    scores = []
    for q in query:
        try:
            scores.append(q_score_b(q, N, doc, len_doc, b))
        except KeyError:
            scores.append(0)
    return sum(scores)


делаем новые матрицы с новыми переменными

In [145]:
bm15_matrix = pd.DataFrame(columns=vectorizer.get_feature_names())
for text in preprocessed_docs[:1000]:
    to_new_m_scores = []
    for item in vectorizer.get_feature_names():
        if item in text.split():
            len_doc = len(text.split())
            to_new_m_scores.append(q_score_b(item, N, text, len_doc, b15))
        else:
            to_new_m_scores.append(0)
    bm15_matrix = bm15_matrix.append(pd.Series(to_new_m_scores, index=bm15_matrix.columns ), ignore_index=True)

In [155]:
bm11_matrix = pd.DataFrame(columns=vectorizer.get_feature_names())
for text in preprocessed_docs[:1000]:
    to_new1_m_scores = []
    for item in vectorizer.get_feature_names():
        if item in text.split():
            len_doc = len(text.split())
            to_new1_m_scores.append(q_score_b(item, N, text, len_doc, b11))
        else:
            to_new1_m_scores.append(0)
    bm11_matrix = bm11_matrix.append(pd.Series(to_new1_m_scores, index=bm11_matrix.columns ), ignore_index=True)

способ поиска матрица*вектор

In [154]:
matrix_search(10, bm15_matrix)



Точность поиска:  0.2


In [156]:
matrix_search(10, bm11_matrix)



Точность поиска:  0.3


а точность поиска с b = 0.75 видно выше, в задании 1 :)